My room bookcase
=============

# Klimpen bookcase
http://www.ikea.com/it/it/catalog/products/10353765/

![klimpen full size](./images/klimpen_1.png)
![klimpen detail](./images/klimpen_2.png)

We will build the various part of the bookcase, then we will put them together. A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).
Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- bottom
Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respec of the dimension and color) Klimpen


Note: measures are in meters

In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.012128 seconds


# Bookcase structure
We will build the various part of the bookcase, then we will put them together.
A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).

Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- base

Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respect of the dimension and color) Klimpen

## Back of the bookcase

In [10]:
def back(width, height):
    """
    back: builds the back of the bookcase
    :param back: height, width in meters
          thickness is fixed
    :return: a CUBOID. The 3d model for the back
    """ 
    return CUBOID([width,height,0.01]) #0.1 is thickness

In [12]:
b = back(0.58, 0.7)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbc660> >

## Shelves

In [4]:
def shelf(width, height, length):
    """
    shelf: builds a single shelf
    :param width: width of the shelf
    :return: a CUBOID. The 3D model for a shelf
    """
    return CUBOID([width,height,length])


def shelves(nshelves, heightBack, width, height, length):
    """
    shelves builds <nshelves> shelves
    :param nshelves: number of shelves
    :param heightBack: height of the back, the area where shelves are positionable
    :param width: width of the shelf.
    :param height: height of the shelf.
    :param length: length of the shelf. (z-axis)
    :return: nshelves CUBOIDs. nshelves shelves
    """
    interval = heightBack/(nshelves+1)
    s = STRUCT(NN(nshelves) ([shelf(width, height, length), T(2)(interval)]))
    return s

In [13]:
ss = shelves(nshelves=2, heightBack=0.58, width=0.68, height=0.02, length=0.3)
VIEW(ss)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbc5d0> >

![2 shelves](./images/2_shelves.png)

## Sides
The lateral components. Two parallel wooden beam

In [14]:
def side(heigth, length):
    """
    side: side of the bookcase structure
    :param heigth: height of the wooden beam
    :param length: length of the wooden beam
    :return: a CUBOID. The 3d model for a lateral wooden beam
    """
    return CUBOID([0.03, heigth, length])

In [7]:
def sides(height, length, distance):
    """
    sides: a couple of sides
    :param height: as side(height,length)
    :param length:  "
    :param distance:  "
    :return: STRUCT([CUBOID,CUBOID])
    """
    s = side(height, length)
    sides = STRUCT([s, T(1)(distance)(s)])
    return sides

In [16]:
single_side = side(0.58, 0.3)
VIEW(single_side)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbc630> >

In [18]:
two_sides = sides(0.58, 0.3, 0.7)
VIEW(two_sides)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1817bbcb40> >

The lateral components of our bookcase

![sides](./images/sides.png)

## Top
The top of our bookcase. It consists of a set of CUBOIDs placed one on top of another

In [52]:
def top(widthSmall, heightSmall, lengthSmall, widthBig, heightBig, lengthBig):
    """
    top: the top of the bookcase, cmade of two different layers/block
    _____________________
    [                   ]
    [      layerBig     ]
    [___________________]
      [ layerSmall    ]
      [_______________]
    
    :param width: the width of the top (layerBig actually), the same of the shelf
    :param height: the height, see above
    :param length: the length, see above
    :param widthBack: the width of the back side of the boockase will be used to calculate the top width
    :param sidesWidth: the width of the two lateral beam, this will be subtracted to widthBack
    :param lengthBack: the length of the back
    :return: a STRUCT made of CUBOIDs (layerBig/small).
    """
    #layer small should have the same dimension of the shelves
    layerSmall = shelf(widthSmall, heightSmall, lengthSmall)
    
    #offset between the two layers
    offX = (widthBig-widthSmall)/2
    offZ = (lengthBig-lengthSmall)/2
    
    layerBig = T([1,2,3])([-offX,heightSmall,-offZ])(CUBOID([widthBig, heightBig, lengthBig]))
    return STRUCT([layerSmall, layerBig])

In [57]:
## Usefull variables
backHeight = 0.58
backWidth = 0.7
backLength = 0.01

shelfHeight = 0.02
shelfLength = 0.3 
shelfWidth = 0.68

bookcaseWidth = shelfWidth+0.04
bookcaseHeight = shelfHeight+0.03
bookcaseLength = shelfLength+0.04

We should keep in mind that the bottom and small layer of the top is the same size as a shelf

In [58]:
t = top(shelfWidth, shelfHeight, shelfLength, bookcaseWidth, bookcaseHeight, bookcaseLength)
VIEW(t)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f67c810> >

![top](./images/top.png)

## Base
The base for our bookcase is similar to the top in the sense that is made of a small layer and a big one. The small one lies upon the big one and is way thinner than the big/bottom one

In [78]:
def base(width, height, length):
    """
    :param width: width of the bookcase
    :param height: height of the bookcase
    :param length: length of the bookcase
    :return: STRUCT. Two CUBOIDs representing the base
    """
    
    layerBig = CUBOID([width, height, length])
    layerSmall = CUBOID([0.96*width, 0.01, 0.96*length])
    
    #offsets. layerSmall has a 0.04^2 smaller perimeter (X,Z)
    offX = width*0.02
    offZ = length*0.02
    
    b = STRUCT([layerBig, T([1, 2, 3])([offX, height, offZ])(layerSmall)])
    return b

In [79]:
b = base(bookcaseWidth, bookcaseHeight, bookcaseLength)
VIEW(b)

![base](./images/base.png)

# Let's build the KLIMPEN bookcase
Now we will create a model for the whole bookcase.
The ggpl_KLIMPEN function takes as reference top/base width and length.

In [ ]:
def ggpl_KLIMPEN(nshelves, width, height, length, color):
    """
    :param nshelves: number of shelves
    :param width: width of the whole bookcase
    :param height: height of the whole bookcase
    :param length: length of the whole bookcase
    :param color: color of the bookcase
    :return: STRUCT. 3d model of Klimpen
    """   
    # base
    baseKlimpen = base(width, height*0.2, length) 
    
    //TODO
    # Top
    topKlimpen = top,width, height*0.1, length)
    
    # sides
    sidesKlimpen = sides(height*0.3, ,)
    